In [1]:
import logging
import os
import datasets.arrow_dataset as da

from transformers import BartForConditionalGeneration, AutoConfig
from transformers.trainer_utils import get_last_checkpoint

import torch

In [103]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [106]:
model = BartForConditionalGeneration(config)

In [107]:
model.load_state_dict(torch.load('./tmp/cache_data/model.pt'))

<All keys matched successfully>

In [4]:
import os
import json
import pandas as pd
from pandas import json_normalize

json_data = []

for filename in os.listdir("tmp/data/Validation"):
   with open(os.path.join("tmp/data/Validation", filename), 'r') as f:
        json_data.append(json.load(f))

In [5]:
from tqdm import tqdm

In [6]:
df = pd.concat([json_normalize(json_data[i]['data']) for i in tqdm(range(len(json_data)))])

100%|██████████| 9/9 [00:08<00:00,  1.03it/s]


In [7]:
dict_data = {'dialogue':df['body.dialogue'], 'summary':df['body.summary'], 'id':df['header.dialogueInfo.dialogueID']}

In [8]:
return_df = pd.DataFrame(data=dict_data)

In [10]:
def generate_summary(test_samples, model, tokenizer, encoder_max_length):
    inputs = tokenizer(
        test_samples["dialogue"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )

    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return outputs, output_str

In [11]:
def flatten(example):
    dialogue_list = []

    for dict_data in example['dialogue']:
        return_string = ""
        for string in dict_data:
            return_string += string['participantID'] + ": " + string['utterance'] + "\r\n"

        dialogue_list.append(return_string[:-2])

    return {
        "dialogue": dialogue_list,
        "summary": example['summary']
    }

In [12]:
import datasets.arrow_dataset as da

In [13]:
sample_dataset = da.Dataset.from_pandas(return_df)

In [15]:
from transformers import PreTrainedTokenizerFast

In [17]:
from typing import Optional

In [18]:
class MyTokenizer(PreTrainedTokenizerFast):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def save_vocabulary(self, save_directory: str, filename_prefix: Optional[str] = None):
        return (1,)

In [19]:
sample_dataset = sample_dataset.map(flatten, remove_columns=['id'], batched=True)

In [20]:
tokenizer = MyTokenizer.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'MyTokenizer'.


In [111]:
encoder_max_length = 1024
decoder_max_length = 128

In [140]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["dialogue"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, 
                             num_beams=5,
                             max_length=64,
                             num_return_sequences=3, #3개의 결과를 디코딩해낸다
                             attention_mask=attention_mask,    
                             top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
                            top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
                            )

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return outputs, output_str

In [ ]:
pred_list = []
golden_list = []

for i in tqdm(range(50)):
    pred_list.append(generate_summary(sample_dataset[10*i:10*(i+1)],model)[1])
    golden_list.append(sample_dataset[10*i:10*(i+1)]['summary'])







  0%|          | 0/50 [00:00<?, ?it/s]





  2%|▏         | 1/50 [00:25<20:47, 25.45s/it]





  4%|▍         | 2/50 [00:52<20:49, 26.03s/it]





  6%|▌         | 3/50 [01:23<21:29, 27.44s/it]





  8%|▊         | 4/50 [01:47<20:19, 26.50s/it]





 10%|█         | 5/50 [02:13<19:34, 26.11s/it]





 12%|█▏        | 6/50 [02:39<19:13, 26.22s/it]





 14%|█▍        | 7/50 [03:07<19:03, 26.59s/it]





 16%|█▌        | 8/50 [03:39<19:45, 28.22s/it]





 18%|█▊        | 9/50 [04:12<20:20, 29.77s/it]





 20%|██        | 10/50 [04:42<19:50, 29.77s/it]





 22%|██▏       | 11/50 [05:12<19:25, 29.89s/it]





 24%|██▍       | 12/50 [05:40<18:41, 29.52s/it]





 26%|██▌       | 13/50 [06:06<17:23, 28.19s/it]





 28%|██▊       | 14/50 [06:33<16:51, 28.10s/it]





 30%|███       | 15/50 [06:59<15:54, 27.27s/it]





 32%|███▏      | 16/50 [07:26<15:22, 27.12s/it]





 34%|███▍      | 17/50 [07:57<15:39, 28.46s/it]





 36%|███▌      | 18/50 [08:25<15:00, 28.14s/it]





 38%|

In [55]:
import nltk

In [109]:
def postprocess_text_first_sent(preds):
    preds = [pred.strip() for pred in preds]
    preds = [pred[:pred.index(".")+1] if "." in pred else pred for pred in preds]
    
    # preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]

    return preds

In [130]:
print(pred_list[0][3])

수유로에서 집 데려다주려고 하는데 집에 데려다주지 않는다.


In [138]:
for i in range(len(pred_list)):
    pred_list[i] = postprocess_text_first_sent(pred_list[i])
    for j in range(len(golden_list[i])):
        print(pred_list[i][j*3:(j+1)*3])
        print(golden_list[i][j])
        print()

['수유로 오라고 해서 지금 가도 되냐고 물으니 집에 데려다주냐고 한다.', '수유로 오라고 해서 지금 가도 되냐고 물으니 안된다고 한다.', '수유로 오라고 해서 지금 가도 되냐고 물으니 집에 데려다주냐고 해서']
지금 수유에 갈 테니 집에 데려다 달라고 한다.

['제주도가 가고 싶어서 2박 3일 동안 갔다 올 수 있을 것 같다고 한다.', '제주도를 가고 싶은데 쉬는 날 몰라서 못 갈 것 같다.', '제주도가 가고 싶어서 2박 3일 동안 갔다 올 수 있을 것 같다고 하자']
2박 3일 정도 쉬는 날을 몰아서 제주도에 갔다 오기로 했다.

['메가박스 2시 45분 매표소나 역출구보다 역출구 매표', '메가박스 2시 45분 매표소나 역출구보다 매표소가 낫지', '메가박스 2시 45분 매표소나 역출구가 나을 것 같다고 하자']
메가박스 매표소에서 2시 45분에 만나기로 약속했다.

['내일 상황을 봐서 밥을 정하고 간지 카페에 가기로 한다.', '내일 상황을 봐서 밥을 정하고 간지 카페에 가자고 한다.', '내일 상황을 봐서 밥을 정하고 간지 카페에 가자고 한다.']
내일 상황을 봐서 정하자고 카페를 가서 힐링해야 한다고 한다.

['내일 모레 김장김치와 수육을 먹으러 가기로 했다.', '내일 모레 김장김치와 수육을 먹으러 가기로 했다.', '내일 모레 김장김치랑 수육을 먹으러 가기로 했다.']
모레 김장 김치와 수육을 먹을 테니 집으로 오라고 한다.

['제주도에서 숙소를 예약하고 있는데 전날까지 고통받는다.', '제주도에서 숙소를 예약하고 있는데 전날까지 고통받는다.', '제주도에서 숙소를 예약하고 있는데 전날까지 고통받는다고 한다.']
동생이 언니에게 자냐며 잘 다녀오라고 하자 언니는 제주도 숙소를 예약하고 있다며 빨리하고 자야겠다고 한다.

['40분쯤에 도착할 것 같으니 시간을 봐서 카페에 가려고 한다.', '40분쯤에 도착할 것 같으니 시간을 봐서 카페에 가려고 한다.', '40분쯤에 도착할 것 같으니 시간을 봐서 모닝커피를 마시라고 한다.']
40